In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [2]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [3]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
lp = StanProblem(bs_model);
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

D = 10;
initial_θ = rand(Uniform(-2, 2), D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.95, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);


  iterations:                                   2
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  1.0
  n_steps:                                      1
  is_accept:                                    true
  acceptance_rate:                              0.0
  log_density:                                  -9.227705930660324
  hamiltonian_energy:                           15.107447958529185
  hamiltonian_energy_error:                     0.0
  max_hamiltonian_energy_error:                 6.730136001714255e34
  tree_depth:                                   0
  numerical_error:                              true
  step_size:                                    5.68859433050041
  nom_step_size:                                5.68859433050041
  is_adapt:                                     true


















Sampling 100%|███████████████████████████████| Time: 0:00:02
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9415382751735039
  log_density:                                  -5.977827095568661
  hamiltonian_energy:                           9.411462823091718
  hamiltonian_energy_error:                     -0.3424740352037823
  max_hamiltonian_energy_error:                 0.5087490101835357
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.3181089727915711
  nom_step_size:                                0.3181089727915711
  is_adapt:                                     false
  mass_matrix:                     

┌ Info: Finished 8000 sampling steps for 1 chains in 2.588498867 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.975567014283514, 0.94245 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.318), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9806533252838789
│   average_acceptance_rate = 0.9473468049872089
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [4]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [5]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [6]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [7]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.323213  0.0841342  -0.0971426  0.0596639  …  0.0918739  4.4095  3.59251